In [2]:
import numpy as np 
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go

In [32]:
sin_portfolio_df = pd.read_csv("sin_portfolio_returnsV2.csv")

other_factors_df = pd.read_csv("final_merged.csv")

# Converting data types for merging

In [41]:
sin_portfolio_df["Date"] = pd.to_datetime(sin_portfolio_df["Date"])
sin_portfolio_df.set_index("Date", inplace=True)
sin_portfolio_df.head()

,Num_Active_Stocks,BEAM,BEER,PFB,ABW,GBL,RUP,CNB,CKB,FAL,...,CEDC,PTT,Portfolio_Returns,Portfolio_Proportion,Unnamed: 97,Risk_Free_Percent,Risk_Free_Decimal,Risk_Free_Monthly,Unnamed: 101,Excess_Return
Date,,,,,,,,,,,,,,,,,,,,,
1962-08-31,19,NaN,NaN,0.005013,NaN,0.005263,0.000248,0.003096,NaN,-0.000405,...,NaN,NaN,-0.028063,0.052632,NaN,3.98,0.0398,0.000274,NaN,-0.02833753
1962-09-30,19,NaN,NaN,-0.002288,NaN,-0.009569,-0.002471,-0.002924,NaN,-0.003672,...,NaN,NaN,7.390793,0.052632,NaN,3.98,0.0398,0.000274,NaN,7.390518496
1962-10-31,19,NaN,NaN,-0.007177,NaN,-0.023392,-0.004148,0.003096,NaN,-0.002193,...,NaN,NaN,-0.095336,0.052632,NaN,3.93,0.0393,0.000271,NaN,-0.095606439
1962-11-30,17,NaN,NaN,NaN,NaN,NaN,0.001258,0.004085,NaN,0.004859,...,NaN,NaN,0.044245,0.058824,NaN,3.92,0.0392,0.000270,NaN,0.043974828
1962-12-31,18,NaN,NaN,NaN,0.0,NaN,0.000873,0.002165,NaN,-0.001562,...,NaN,NaN,2.401140,0.055556,NaN,3.86,0.0386,0.000266,NaN,2.400873839


In [42]:
other_factors_df.rename(columns = {"Unnamed: 0": "Date"}, inplace = True)
other_factors_df["Date"] = pd.to_datetime(other_factors_df["Date"])
other_factors_df.set_index("Date", inplace=True)
other_factors_df.head()

,ABEV,ABUD,ABV,ABW,ACCOB,ADC,ADSO,AED,AFTI,BACA,...,VO,FARM,GMCR,JVA,PEET,Excess Return on the Market,SMB,HML,MOM,USREC
Date,,,,,,,,,,,,,,,,,,,,,
1962-07-31,NaN,NaN,NaN,NaN,NaN,37.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0628,0.0163,-0.0357,0.0055,0
1962-08-31,NaN,NaN,NaN,NaN,NaN,37.250,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0213,0.0125,-0.0121,-0.0058,0
1962-09-30,NaN,NaN,NaN,NaN,NaN,37.250,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.0522,-0.0248,0.0134,0.0395,0
1962-10-31,NaN,NaN,NaN,NaN,NaN,33.375,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.0005,-0.0401,0.0135,0.0075,0
1962-11-30,NaN,NaN,NaN,2.5,NaN,39.625,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.1087,0.0259,0.0099,-0.0717,0


# Filtering required columns only

In [43]:
sin_portfolio_df = sin_portfolio_df[["Portfolio_Returns", "Excess_Return"]]
other_factors_df = other_factors_df[["Excess Return on the Market", "SMB", "HML", "MOM", "USREC"]]


# Merging for model building

## Checking shapes before merging

In [45]:
sin_portfolio_df.shape, other_factors_df.shape

((701, 2), (702, 5))

In [47]:
pd.concat([sin_portfolio_df, other_factors_df], axis = 1, join = "inner")

,Portfolio_Returns,Excess_Return,Excess Return on the Market,SMB,HML,MOM,USREC
Date,,,,,,,
1962-08-31,-0.028063,-0.02833753,0.0213,0.0125,-0.0121,-0.0058,0
1962-09-30,7.390793,7.390518496,-0.0522,-0.0248,0.0134,0.0395,0
1962-10-31,-0.095336,-0.095606439,-0.0005,-0.0401,0.0135,0.0075,0
1962-11-30,0.044245,0.043974828,0.1087,0.0259,0.0099,-0.0717,0
1962-12-31,2.401140,2.400873839,0.0101,-0.0380,0.0035,0.0588,0
...,...,...,...,...,...,...,...
2020-08-31,-0.132603,-0.132648155,0.0763,-0.0022,-0.0293,0.0051,0
2020-09-30,9.329493,9.329446064,-0.0363,-0.0004,-0.0266,0.0305,0
2020-10-31,-0.116476,-0.116531269,-0.0210,0.0439,0.0419,-0.0303,0


# Model Building